**Calpine Energy - Power Output for 2021 using ARIMA**

Srushti Padade


In [1]:
# Libraries

import pandas as pd
import seaborn as sns
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Loading the Dataset

df1 = pd.read_csv("/content/drive/MyDrive/Machine Data/Freestone_CTG1_2017.csv")
df2 = pd.read_csv("/content/drive/MyDrive/Machine Data/Freestone_CTG1_2018.csv")
df3 = pd.read_csv("/content/drive/MyDrive/Machine Data/Freestone_CTG1_2019.csv")
df4 = pd.read_csv("/content/drive/MyDrive/Machine Data/Freestone_CTG1_2020.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Removing the unrelated columns

cols = [1,4,5]
df1 = df1.drop(df1.columns[cols], axis = 1)
df2 = df2.drop(df2.columns[cols], axis = 1)
df3 = df3.drop(df3.columns[cols], axis = 1)
df4 = df4.drop(df4.columns[cols], axis = 1)

In [4]:
# Renaming the Dataframe

frames = [df1, df2, df3, df4]
for x in frames:
  x.columns = ['Attribute', 'Time', 'Value']

In [5]:
# Concating the dataframes for Training dataset

frame1 = [df1, df2, df3]
Data_df = pd.concat(frame1)
df_train = Data_df
train = df_train[df_train['Attribute']=='Gross Power Output - KPI']

In [6]:
# Test Dataset

df_test = df4
test = df_test[df_test['Attribute']=='Gross Power Output - KPI']

In [7]:
# Removing Duplicate

train = train.drop_duplicates(subset = ['Time'], keep = 'last').reset_index(drop = True)
test  = test.drop_duplicates(subset = ['Time'], keep = 'last').reset_index(drop = True)

In [8]:
# Converting Time variable into Hour and Date

train = train.set_index('Time')
train.index = pd.to_datetime(train.index, format='%Y-%m-%d %H:%M:%S')
train['Date'] = train.index.date
train['HourOfDay'] = train.index.hour

test = test.set_index('Time')
test.index = pd.to_datetime(test.index, format='%Y-%m-%d %H:%M:%S')
test['Date'] = test.index.date
test['HourOfDay'] = test.index.hour

In [9]:
del train['Attribute']
train

Value        Date  HourOfDay
Time                                                    
2017-01-03 21:25:00  1.600000e-07  2017-01-03         21
2017-01-03 21:35:00  1.600000e-07  2017-01-03         21
2017-01-03 00:05:00  1.600000e-07  2017-01-03          0
2017-01-03 03:55:00  1.600000e-07  2017-01-03          3
2017-01-03 08:00:00  1.600000e-07  2017-01-03          8
...                           ...         ...        ...
2019-12-29 23:40:00  1.608975e+02  2019-12-29         23
2019-12-29 23:55:00  1.619234e+02  2019-12-29         23
2019-12-29 23:05:00  1.581167e+02  2019-12-29         23
2019-12-29 23:45:00  1.611879e+02  2019-12-29         23
2019-12-29 23:50:00  1.615912e+02  2019-12-29         23

[315343 rows x 3 columns]

In [10]:
del test['Attribute']
test

Value        Date  HourOfDay
Time                                                  
2020-01-02 00:05:00  159.796000  2020-01-02          0
2020-01-02 00:15:00  159.730000  2020-01-02          0
2020-01-02 00:30:00  159.782067  2020-01-02          0
2020-01-02 00:50:00  159.648600  2020-01-02          0
2020-01-02 00:55:00  159.664000  2020-01-02          0
...                         ...         ...        ...
2020-12-03 16:00:00  122.860933  2020-12-03         16
2020-12-03 16:45:00  155.134200  2020-12-03         16
2020-12-03 04:15:00  182.612200  2020-12-03          4
2020-12-03 00:10:00  181.019400  2020-12-03          0
2020-12-03 04:20:00  182.747133  2020-12-03          4

[97428 rows x 3 columns]

In [11]:
train = train.sort_values(["Time"])
test = test.sort_values(['Time'])

In [12]:
# Aggregating the Data in Hourly manner

MeanAgg_df = train.groupby(['Date', 'HourOfDay'])[['Value']].mean().reset_index()
Test_MeanDf = test.groupby(['Date', 'HourOfDay'])[['Value']].mean().reset_index()

In [13]:
Test_MeanDf

Date  HourOfDay       Value
0     2020-01-01          0  161.361789
1     2020-01-01          1  162.726828
2     2020-01-01          2  163.662806
3     2020-01-01          3  163.372711
4     2020-01-01          4  164.064489
...          ...        ...         ...
8114  2020-12-04          2  182.673556
8115  2020-12-04          3  182.645689
8116  2020-12-04          4  182.669094
8117  2020-12-04          5  182.681011
8118  2020-12-04          6  182.679850

[8119 rows x 3 columns]

In [14]:
MeanAgg_df = MeanAgg_df.fillna(0)

In [15]:
dti = pd.date_range("2017-01-01", periods=26280, freq="H")
dti = pd.DataFrame(dti)

MeanAgg_df['Time'] = dti
MeanAgg_df = MeanAgg_df.set_index('Time')

In [ ]:
# Checking if the data is stationary 

from statsmodels.tsa.stattools import adfuller
from numpy import log
result = adfuller(MeanAgg_df['Value'])
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


ADF Statistic: -11.768087
p-value: 0.000000


In [16]:
MeanAgg_df.tail(10)

Date  HourOfDay       Value
Time                                                  
2019-12-31 14:00:00  2019-12-31         14  166.456622
2019-12-31 15:00:00  2019-12-31         15  164.019083
2019-12-31 16:00:00  2019-12-31         16  163.069233
2019-12-31 17:00:00  2019-12-31         17  162.634611
2019-12-31 18:00:00  2019-12-31         18  161.708533
2019-12-31 19:00:00  2019-12-31         19  161.891133
2019-12-31 20:00:00  2019-12-31         20  160.427033
2019-12-31 21:00:00  2019-12-31         21  160.274989
2019-12-31 22:00:00  2019-12-31         22  159.977133
2019-12-31 23:00:00  2019-12-31         23  160.577061

In [17]:
dti2 = pd.date_range("2020-01-01", periods=8118, freq="H")
dti2 = pd.DataFrame(dti2)

Test_MeanDf['Time'] = dti2
Test_MeanDf = Test_MeanDf.set_index('Time')

In [18]:
del MeanAgg_df['Date']
del MeanAgg_df['HourOfDay']

# Modeling

In [ ]:
# Training Seasonal ARIMA Model on Training dataset (2017 - 19)

import statsmodels.api as sm

mod = sm.tsa.statespace.SARIMAX(MeanAgg_df['Value'],
                                order=(1, 0, 2),
                                seasonal_order=(2, 1, 0, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)


In [ ]:
# Fitting the model

results = mod.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                 
==========================================================================================
Dep. Variable:                              Value   No. Observations:                26280
Model:             SARIMAX(1, 0, 2)x(2, 1, 0, 12)   Log Likelihood             -111009.121
Date:                            Wed, 24 Feb 2021   AIC                         222030.241
Time:                                    20:04:56   BIC                         222079.292
Sample:                                01-01-2017   HQIC                        222046.081
                                     - 12-31-2019                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.9042      0.003    358.257      0.000       0.899       0.909
ma.L1          0.2376      0.004     53.882      0.000       0.229       0.246
ma.L2          0.0201      0.006      3.621      0.000       0.009       0.031
ar.S.L12      -0.7045      0.003   -240.323      0.000      -0.710      -0.699
ar.S.L24      -0.2847      0.003    -96.922      0.000      -0.290      -0.279
sigma2       276.4991      0.773    357.725      0.000     274.984     278.014
===================================================================================
Ljung-Box (Q):                     2934.28   Jarque-Bera (JB):            603572.77
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.38   Skew:                            -0.77
Prob(H) (two-sided):                  0.00   Kurtosis:                        26.44
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [ ]:
# Forecasting the Gross Power Output for 2020

pred_uc = results.get_forecast(steps=8118)
pred = results.get_prediction(start=pd.to_datetime("2020-01-01"), end = pd.to_datetime("2020-12-31") , dynamic=False)
data = pd.DataFrame(pred.predicted_mean)
data['forcast'] = data

In [ ]:
# Comparision with test dataset(2020) and predicted value of 2020.

Test_analysis = data.drop(data.columns[0], axis = 1)
Test_analysis['org'] = Test_MeanDf['Value']
Test_analysis = Test_analysis.reset_index()

In [ ]:
import matplotlib.pyplot as plt

# gca stands for 'get current axis'
ax = plt.gca()

Test_analysis.plot(kind='line',x='index',y='forcast',ax=ax)
Test_analysis.plot(kind='line',x='index',y='org', color='red', ax=ax)

plt.show()

In [ ]:
Test_analysis = Test_analysis.set_index('index')
Test_analysis.head(25)

forcast         org
index                                      
2020-01-01 00:00:00  161.448162  161.361789
2020-01-01 01:00:00  161.924864  162.726828
2020-01-01 02:00:00  162.213324  163.662806
2020-01-01 03:00:00  161.852094  163.372711
2020-01-01 04:00:00  162.126905  164.064489
2020-01-01 05:00:00  161.979265  164.516100
2020-01-01 06:00:00  161.898893  165.298322
2020-01-01 07:00:00  161.859237  166.516817
2020-01-01 08:00:00  161.328042  154.676477
2020-01-01 09:00:00  161.774338  101.912588
2020-01-01 10:00:00  161.846817   90.454192
2020-01-01 11:00:00  162.120006  118.214592
2020-01-01 12:00:00  162.833118  110.962353
2020-01-01 13:00:00  163.051317  137.346528
2020-01-01 14:00:00  163.473817  163.548283
2020-01-01 15:00:00  162.719293  162.646772
2020-01-01 16:00:00  162.869270  160.940061
2020-01-01 17:00:00  162.656037  160.680156
2020-01-01 18:00:00  162.509574  159.584861
2020-01-01 19:00:00  162.531564  158.675467
2020-01-01 20:00:00  161.851506  158.672167
2020-01-01 21:00:00  162.094934  158.748800
2020-01-01 22:00:00  162.081451  159.250278
2020-01-01 23:00:00  162.393056  159.771800
2020-01-02 00:00:00  163.069104  159.721200

In [ ]:
# Accuracy

rmse = np.mean((Test_analysis['forcast'] - Test_analysis['org'])**2)**.5
rmse

88.51005258862892

In [ ]:
# Forecasting using 2017-19 training model

forcast = results.get_prediction(start=pd.to_datetime("2021-01-01"), end = pd.to_datetime("2021-12-31") , dynamic=False)
Forcast_2021 = pd.DataFrame(forcast.predicted_mean)
Forcast_2021

0
2021-01-01 00:00:00  162.699130
2021-01-01 01:00:00  162.957101
2021-01-01 02:00:00  163.305426
2021-01-01 03:00:00  162.463559
2021-01-01 04:00:00  162.513226
...                         ...
2021-12-30 20:00:00  161.380852
2021-12-30 21:00:00  161.647319
2021-12-30 22:00:00  161.630626
2021-12-30 23:00:00  161.976271
2021-12-31 00:00:00  162.699130

[8737 rows x 1 columns]

In [ ]:
Forcast_2021.to_csv('Forcast.csv')

In [20]:
del Test_MeanDf['Date']
del Test_MeanDf['HourOfDay']

In [21]:
# Combining training and test data (2017 - 2020)

Total_datadf = MeanAgg_df.append(Test_MeanDf)
Total_datadf = Total_datadf.loc[Total_datadf.index.notnull()]
Total_datadf

Value
Time                             
2017-01-01 00:00:00  1.600000e-07
2017-01-01 01:00:00  1.600000e-07
2017-01-01 02:00:00  1.600000e-07
2017-01-01 03:00:00  1.600000e-07
2017-01-01 04:00:00  1.600000e-07
...                           ...
2020-12-04 01:00:00  1.827487e+02
2020-12-04 02:00:00  1.826736e+02
2020-12-04 03:00:00  1.826457e+02
2020-12-04 04:00:00  1.826691e+02
2020-12-04 05:00:00  1.826810e+02

[34398 rows x 1 columns]

In [22]:
# Modeling Seasonal ARIMA using entire dataset 

import statsmodels.api as sm

mod = sm.tsa.statespace.SARIMAX(Total_datadf['Value'],
                                order=(1, 0, 2),
                                seasonal_order=(2, 1, 0, 12),
                                enforce_stationarity=False,
                                enforce_invertibility=False)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency H will be used.
  % freq, ValueWarning)


In [23]:
results = mod.fit()
pred_uc = results.get_forecast(steps=8736)

In [24]:
forcast = results.get_prediction(start=pd.to_datetime("2021-01-01"), end = pd.to_datetime("2021-12-31") , dynamic=False)
Forcast_2021 = pd.DataFrame(forcast.predicted_mean)

In [25]:
Forcast_2021

0
2021-01-01 00:00:00  188.294464
2021-01-01 01:00:00  187.709686
2021-01-01 02:00:00  174.382792
2021-01-01 03:00:00  165.423985
2021-01-01 04:00:00  168.935260
...                         ...
2021-12-30 20:00:00  174.600600
2021-12-30 21:00:00  165.110865
2021-12-30 22:00:00  175.854842
2021-12-30 23:00:00  176.361673
2021-12-31 00:00:00  188.294464

[8737 rows x 1 columns]

In [26]:
Forcast_2021.to_csv('Forcast.csv')